In [15]:
import pandas as pd
import os


def load_and_combine_pkl_files(directory_path):
    # List to hold all the dataframes
    dataframes = []

    # Loop through all the files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith('perturbed_inferenced_df.pkl'):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Load the dataframe from a pkl file
            df = pd.read_pickle(file_path)
            # Append the dataframe to the list
            dataframes.append(df)

    # Concatenate all dataframes into one big dataframe
    big_df = pd.concat(dataframes, ignore_index=True)

    return big_df


# Usage
directory_path = '/Users/ximing/Desktop/Explainprompt'
big_df = load_and_combine_pkl_files(directory_path)
print(big_df)


                                                prompt real_output  \
0    maintains your sympathy for this otherwise cha...  ^POSITIVE^   
1    all that memorable , but as downtown saturday ...  ^NEGATIVE^   
2    there 's lots of cool stuff packed into espn '...  ^POSITIVE^   
3    that this vapid vehicle is downright doltish a...  ^NEGATIVE^   
4    a laughable -- or rather , unlaughable -- excu...  ^NEGATIVE^   
..                                                 ...         ...   
322  suffers from too much norma rae and not enough...  ^NEGATIVE^   
323  offering next to little insight into its intri...  ^NEGATIVE^   
324  a smart , provocative drama that does the near...  ^POSITIVE^   
325  what passes for sex in the movies look like ch...  ^NEGATIVE^   
326  gloriously flippant as lock , stock and two sm...  ^POSITIVE^   

                                           token_level  \
0    {'tokens': [{'token': 'maint', 'type': 'input'...   
1    {'tokens': [{'token': 'all', 'type': '

In [16]:
print(big_df.columns)

Index(['prompt', 'real_output', 'token_level', 'word_level', 'label',
       'component_level', 'instruction', 'query', 'component_range',
       'instruction_weight', 'query_weight', 'instructions_tokens',
       'query_tokens', 'instruction_token_top_0.2_peturbed',
       'instruction_token_bottom_0.2_peturbed', 'query_token_top_0.2_peturbed',
       'query_token_bottom_0.2_peturbed', 'top_reconstructed_instruction_0.2',
       'top_reconstructed_query_0.2', 'bottom_reconstructed_instruction_0.2',
       'bottom_reconstructed_query_0.2',
       'top_reconstructed_instruction_0.2_result',
       'top_reconstructed_query_0.2_result',
       'bottom_reconstructed_instruction_0.2_result',
       'bottom_reconstructed_query_0.2_result'],
      dtype='object')


In [17]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.07033639143730887


In [18]:
differences = big_df['real_output'] != big_df['top_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.3363914373088685


In [19]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

0.1529051987767584


In [20]:
differences = big_df['real_output'] != big_df['top_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

0.9877675840978594


In [21]:
print(big_df['instruction_weight'].mean())

0.796419031248912


In [22]:
print(big_df['query_weight'].mean())

0.20105899439910707
